In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import timedelta,datetime
df=pd.read_csv('C:\\workplace\\electricity_demand\\dataset\\AEP_hourly.csv')
df.columns=['date_time','aep_mw']
df['date_time']=pd.to_datetime(df['date_time'])
df.sort_values('date_time',ascending=True,inplace=True)
df['date_time']=df['date_time'].astype(str)

In [22]:
def preparion_train (df,num_lags,lag_feature):

    df_lags=pd.DataFrame(index=df.index,columns=['lags_{}'.format(671-i) for i in range(num_lags)])
    for i in range(num_lags):
        df_lags['lags_{}'.format(671-i)]=df[lag_feature].shift(671-i)
    
    df_targets=pd.DataFrame(index=df.index,columns=['step_{}'.format(i) for i in range(16,40)])
    for i in range(16,40):
        df_targets['step_{}'.format(i)]=df[lag_feature].shift(-i)
    
    df_processed=pd.concat([df[['date_time']],df_targets,df_lags],axis=1)
    #df_processed=df_processed[pd.to_datetime(df_processed['date_time']).dt.hour==trigerred_time]
    return df_processed.dropna()

# Train and Test split

In [23]:
from sklearn.preprocessing import PowerTransformer
scaler=PowerTransformer().fit(df.loc[df['date_time']<='2017-08-02 23:00:00',['aep_mw']])
df['scaled_aep_mw']=pd.Series(scaler.transform(df[['aep_mw']]).reshape(-1),index=df.index)
df.head()

,date_time,aep_mw,scaled_aep_mw
2183,2004-10-01 01:00:00,12379.0,-1.276695
2184,2004-10-01 02:00:00,11935.0,-1.492740
2185,2004-10-01 03:00:00,11692.0,-1.614218
2186,2004-10-01 04:00:00,11597.0,-1.662359
2187,2004-10-01 05:00:00,11681.0,-1.619773


In [24]:
df_processed=preparion_train (df,24*7*4,'scaled_aep_mw')
df_processed

,date_time,step_16,step_17,step_18,step_19,step_20,step_21,step_22,step_23,step_24,...,lags_9,lags_8,lags_7,lags_6,lags_5,lags_4,lags_3,lags_2,lags_1,lags_0
1558,2004-10-29 00:00:00,-0.105342,-0.080372,-0.156780,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,...,-0.030725,-0.076813,-0.122441,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436
1511,2004-10-29 01:00:00,-0.080372,-0.156780,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,...,-0.076813,-0.122441,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351
1512,2004-10-29 02:00:00,-0.156780,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,-1.622300,...,-0.122441,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351,-1.449815
1513,2004-10-29 03:00:00,-0.092646,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,-1.622300,-1.816869,...,-0.170407,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351,-1.449815,-1.583501
1514,2004-10-29 04:00:00,0.047289,-0.055106,-0.221166,-0.521421,-0.948255,-1.362899,-1.622300,-1.816869,-1.954480,...,-0.097404,0.115540,0.094026,-0.011523,-0.397106,-0.826436,-1.214351,-1.449815,-1.583501,-1.696536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116166,2018-08-01 05:00:00,0.422499,0.326766,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,...,0.501312,0.458576,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001
116167,2018-08-01 06:00:00,0.326766,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,...,0.458576,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482
116168,2018-08-01 07:00:00,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,-0.691662,...,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482,-0.756927
116169,2018-08-01 08:00:00,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,-0.691662,-0.408344,...,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482,-0.756927,-0.486737


In [27]:
X_train=df_processed.loc[df_processed['date_time']<='2017-08-02 23:00:00',df_processed.columns[df_processed.columns.str.contains('lags_')]]
X_test=df_processed.loc[(df_processed['date_time']>='2017-08-02 08:00:00') &\
    (pd.to_datetime(df_processed['date_time']).dt.hour==8),df_processed.columns[df_processed.columns.str.contains('lags_')]]
X_train=np.array(X_train).reshape(X_train.shape[0],1,X_train.shape[1])
X_test=np.array(X_test).reshape(X_test.shape[0],1,X_test.shape[1])

y_train=df_processed.loc[df_processed['date_time']<='2017-08-02 23:00:00',df_processed.columns[df_processed.columns.str.contains('step_')]]
y_test=df_processed.loc[(df_processed['date_time']>='2017-08-02 08:00:00') &\
    (pd.to_datetime(df_processed['date_time']).dt.hour==8),df_processed.columns[df_processed.columns.str.contains('step_')]]

# Hyperparameter tuning

In [31]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Input,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [32]:
from sklearn.metrics import mean_absolute_percentage_error

In [33]:
import optuna

In [35]:
from sklearn.model_selection import train_test_split 
X_fit,X_val,y_fit,y_val=train_test_split(X_train,y_train,test_size=0.3,shuffle=False)

In [16]:
stoppage_rule=EarlyStopping(patience=20,monitor='val_loss')

In [52]:
def val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s):

    # Input layer
    model=Sequential()
    #model.add(Input(X_fit.shape[0],X_fit.shape[1],X_fit.shape[2]))
    # Hidden Layers
    for i in range(num_layers):
        if i==0:
            model.add(LSTM(units=num_nerons,return_sequences=True,input_shape=(X_fit.shape[1],X_fit.shape[2])))
        elif i<num_layers-1:
            model.add(LSTM(units=num_nerons,return_sequences=True))
        else :
            model.add(LSTM(units=num_nerons))
    # output layer
    model.add(Dense(units=24))
    # Training
    model.compile(optimizer=Adam(learning_rate=alpha),loss='mse')
    model.fit(X_fit,y_fit,validation_data=(X_val,y_val),epochs=iter,batch_size=b_s,callbacks=[stoppage_rule])
    # Evaluation of the model
    pred=model.predict(X_val)
    y_val2=scaler.inverse_transform(np.array(y_val).reshape(-1,1))
    pred2=scaler.inverse_transform(pred.reshape(-1,1))
    error=mean_absolute_percentage_error(y_val2,pred2)
    return error

In [57]:
def objective(trial):

    num_layers=trial.suggest_int('num_layers',1,4)
    num_nerons=trial.suggest_int('num_nerons',32,256,32)
    alpha=trial.suggest_float('alpha',0.001,0.1)
    iter=trial.suggest_int('iter',1000,1000)
    b_s=trial.suggest_int('b_s',32,1024)
    loss_fun=val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s)
    return loss_fun 
study = optuna.create_study()
study.optimize(objective,n_trials=50,n_jobs=-1)

[I 2024-02-17 12:45:55,296] A new study created in memory with name: no-name-844711af-0050-4a80-a164-f029edd32755


Epoch 1/1000
Epoch 1/1000
Epoch 1/1000
Epoch 1/1000
Epoch 1/1000
Epoch 1/1000
Epoch 1/1000
Epoch 1/1000
83/83 [==============================] - 19s 225ms/step - loss: 0.9511 - val_loss: 1.1536
Epoch 5/1000
141/141 [==============================] - 119s 399ms/step - loss: 0.2557 - val_loss: 0.2897
Epoch 2/1000
144/144 [==============================] - 120s 395ms/step - loss: 0.2195 - val_loss: 0.2463
Epoch 2/1000
85/85 [==============================] - 105s 372ms/step - loss: 0.2482 - val_loss: 0.2725
Epoch 2/1000
83/83 [==============================] - 28s 343ms/step - loss: 0.9509 - val_loss: 1.1543
Epoch 6/1000
580/580 [==============================] - 144s 210ms/step - loss: 0.9599 - val_loss: 1.1815
Epoch 2/1000
85/85 [==============================] - 28s 334ms/step - loss: 0.1727 - val_loss: 0.2643
Epoch 3/1000
141/141 [==============================] - 43s 309ms/step - loss: 0.2016 - val_loss: 0.2711
Epoch 3/1000
144/144 [==============================] - 62s 430ms/step - 

[I 2024-02-17 12:53:37,752] Trial 2 finished with value: 0.0654591129690065 and parameters: {'num_layers': 2, 'num_nerons': 192, 'alpha': 0.013101613178567003, 'iter': 1000, 'b_s': 922}. Best is trial 2 with value: 0.0654591129690065.


141/141 [==============================] - 67s 479ms/step - loss: 0.1807 - val_loss: 0.2792
Epoch 6/1000
83/83 [==============================] - 28s 338ms/step - loss: 0.9508 - val_loss: 1.1592
Epoch 18/1000
144/144 [==============================] - 53s 365ms/step - loss: 0.0693 - val_loss: 0.2499
Epoch 9/1000
83/83 [==============================] - 31s 379ms/step - loss: 0.1577 - val_loss: 0.2532
Epoch 14/1000
580/580 [==============================] - 185s 320ms/step - loss: 0.9533 - val_loss: 1.1455
Epoch 4/1000
141/141 [==============================] - 68s 483ms/step - loss: 0.1764 - val_loss: 0.2828
Epoch 7/1000
141/141 [==============================] - 56s 401ms/step - loss: 0.1466 - val_loss: 0.2613
Epoch 10/1000
83/83 [==============================] - 26s 313ms/step - loss: 0.9510 - val_loss: 1.1674
Epoch 20/1000
144/144 [==============================] - 52s 357ms/step - loss: 0.0649 - val_loss: 0.2513
Epoch 10/1000
83/83 [==============================] - 34s 418ms/step